In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print('Setup complete.')

In [ ]:
# Load all of the Kaggle forum messages, see how many there are, and look at the first few.
forum_message_df = pd.read_csv('/kaggle/input/meta-kaggle/ForumMessages.csv')
print(forum_message_df.shape)
forum_message_df.head()

In [ ]:
# Remove messages that have not been upvoted (Medal == NaN)
forum_message_df = forum_message_df[forum_message_df['Medal'].notna()]
print(forum_message_df.shape)
forum_message_df.head()

In [ ]:
# Remove messages that do not have any message body.
forum_message_df = forum_message_df[forum_message_df['Message'].notna()]
print(forum_message_df.shape)
forum_message_df.head()

In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse

urls = list()
domains = list()

# Loop through the message values and extract URLs from anchor tags.
for message in forum_message_df['Message'].values:
    soup = BeautifulSoup(message)

    for a in soup.find_all('a', href=True):
        urls.append(a['href'])
        domain = urlparse(a['href']).netloc
        domains.append(domain)
        
# Show how many URLs and domains were extracted
print("URLs:", len(urls))
print("domains:", len(domains))

In [ ]:
# Display the first 10 urls and domains.
print(urls[0:10])
print(domains[0:10])

In [ ]:
from collections import Counter

url_counter = Counter(urls)
domain_counter = Counter(domains)

# Find the most common URLs and domains.
top_urls = url_counter.most_common(25)
top_domains = domain_counter.most_common(25)

print("Most common URLs:")
print(top_urls)

print("Most common domains:")
print(top_domains)

Those results are somewhat interesting, but all of the internal links are overwhelming. Let's look at just the external links. We can do that without looping over the dataframe again (slow!) by using the results already stored in the urls and domains lists.

In [ ]:
external_urls = list()
external_domains = list()

domains_to_exclude = ['www.kaggle.com', '', 'blog.kaggle.com']

for url, domain in zip(urls, domains):
    if domain not in domains_to_exclude:
        external_urls.append(url)
        external_domains.append(domain)
        
# Display the first 10 remaining urls and domains.
print(external_urls[0:10])
print(external_domains[0:10])

In [ ]:
external_url_counter = Counter(external_urls)
external_domain_counter = Counter(external_domains)

# Find the most common external URLs and domains.
top_external_urls = external_url_counter.most_common(25)
top_external_domains = external_domain_counter.most_common(25)

print("Most common external URLs:")
print(top_external_urls)

print("Most common external domains:")
print(top_external_domains)

This looks a lot better, but there are still a few odd results. In particular, I'd like to take a closer look at that medium.com URL (second result) that appears to be broken. Let's find all the posts that contain that partial link.

In [ ]:
medium_df = forum_message_df[forum_message_df['Message'].str.contains('https://medium.com/<a href=')]
medium_df

By going directly to several of these posts, I see that these are all links to different articles on Medium. I'm not sure how the link got mangled in the Meta Kaggle dataset, but I'm going to just ignore this particular result.

### Top External Resources
These are the most frequently linked individual pages (and how many times they've been shared) after taking the top 25, then eliminating duplicates.
1. [ML Wave Kaggle Ensembling Guide](https://mlwave.com/kaggle-ensembling-guide/) (120)
2. [Coursera Machine Learning](https://www.coursera.org/learn/machine-learning) (110)
3. [All MOOC BLOG](https://drive.google.com/file/d/1J5NTL7bKW9vkx-S07CqVE2A1Dulc4gTM/view?usp=sharing) (70)
4. [Coursera Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning) (68)
5. [Pretrained models for Pytorch](https://github.com/Cadene/pretrained-models.pytorch) (67)
6. [Kaggle Etiquette - YouTube](https://www.youtube.com/watch?v=aIus8si_Et0) (66)
7. [Kaggle Lingo - YouTube](https://www.youtube.com/watch?v=sEJHyuWKd-s) (62)
8. [Kaggle/docker-python](https://github.com/Kaggle/docker-python) (54)
9. [Kaggle API](https://github.com/Kaggle/kaggle-api) (54)
10. [How to Make a Prediction - Intro to Deep Learning #1](https://www.youtube.com/watch?v=vOppzHpvTiQ&list=PL2-dafEMk2A7YdKv4XfKpfbTH5z6rEEj3) (53)
11. [How to Enter a Kaggle Competition (using Kernels) - YouTube](https://www.youtube.com/watch?&v=GJBOMWpLpTQ) (47)
12. [Image Segmentation Models](https://github.com/qubvel/segmentation_models.pytorch) (46)
13. [Statistics and probability - Khan Academy](https://www.khanacademy.org/math/statistics-probability) (46)
14. [Deep Learning (MIT Press textbook)](https://www.deeplearningbook.org/) (45)
15. [Practical Deep Learning for Coders](http://course.fast.ai/) (44)
16. [PyTorch Image Models](https://github.com/rwightman/pytorch-image-models) (43)
17. [EfficientNet PyTorch](https://github.com/lukemelas/EfficientNet-PyTorch) (42)
18. [Springer Ebooks](https://drive.google.com/file/d/1XRCbtNz2k-H5b_CXO-ZSAxnoD6S2NZTF/view?usp=drivesdk) (40)
19. [MEGA Data Science books (R)](https://mega.nz/folder/0iZFXCbA#Rwh3Km42_YaRvgY_NOAvWw) (40)
20. [MEGA Data Science books](https://mega.nz/folder/g2BRhaDJ#v2XWSegTk3sH6ZcLPNG-WA) (40)
21. [MEGA Python books](https://mega.nz/folder/NmQRlaBa#0FKTDkkHYBmkSmcEu0kGoQ) (40)
22. [Chai Time Data Science Show](https://www.youtube.com/playlist?list=PLLvvXm0q8zUbiNdoIazGzlENMXvZ9bd3x) (39)
23. [TOP 10 Best Online Learning Platforms](https://drive.google.com/file/d/1J0ct16O9ULpqgrmEVazhiuHGlSy2aQmI/view?usp=sharing) (38)

### Top External Sites
These are the most frequently linked sites by the Kaggle community.
1. [GitHub](github.com) (13,428)
2. [YouTube](www.youtube.com) (4,076)
3. [arXiv.org](arxiv.org) (3,715)
4. [Towards Data Science](towardsdatascience.com) (1,713)
5. [Wikipedia](en.wikipedia.org) (1,447)
6. [Medium](medium.com) (1,294)
7. [LinkedIn](www.linkedin.com) (1,042)
8. [scikit-learn](scikit-learn.org) (875)
9. [Coursera](www.coursera.org) (798)
10. [Google Drive](drive.google.com) (789)
11. [Analytics Vidhya](www.analyticsvidhya.com) (655)
12. [OReilly](www.oreilly.com) (609)
13. [Stack Overflow](stackoverflow.com) (566)
14. [Machine Learning Mastery](machinelearningmastery.com) (535)
15. [Jupyter Notebooks nbviewer](nbviewer.jupyter.org) (420)
16. [TensorFlow](www.tensorflow.org) (419)
17. [Twitter](twitter.com) (419)
18. [edX](www.edx.org) (330)
19. [Google Cloud](cloud.google.com) (319)
20. [Cognitive Class](cognitiveclass.ai) (311)
21. [Data Science Central](www.datasciencecentral.com) (294)
22. [Udemy](www.udemy.com) (275)
23. [MEGA](mega.nz) (275)

I've also created a post on the Kaggle General forum to share these results: [Most Useful ML/Learning Resources](https://www.kaggle.com/general/263231)